### Preprocessing e esecuzione esperimenti

In [1]:
import pandas as pd
from datetime import datetime
import math
from utils.scalability_utils import cluster_to_dataframe
from models.SSPMF import SSPMFModel
from models.JGSMF_3 import solve_with_phases as solve_with_phases_3
from models.JGSMF_2 import solve_with_phases as solve_with_phases_2
from models.JGSMF_1 import solve_with_phases as solve_with_phases_1
from models.JGSMF import solve_with_phases as solve_with_phases_0

In [2]:
cluster_directory = "../data/MTSP/Laporte/Tabela4/"

df = cluster_to_dataframe(cluster_directory)

df = df.sample(frac=1).reset_index(drop=True)

In [3]:
print(df['num_tools'].unique())
print(df['magazine_capacity'].unique())


[20 25 15]
[10 20  5 15]


In questo dataset tutti le istanze hanno 9 job e un numero di tools tra 15, 20 o 25. La capienza invece è tra 5, 10, 15 o 20.
Per l'analisi di scalabilità ho deciso di prendere tutte le combinazioni di numero di tools e capienza, per poi valutarne la velocità di esecuzione tra i 2 modelli

In [ ]:
scalability_df = df.drop_duplicates(subset=['num_tools', 'magazine_capacity'])

Ogni esperimento su queste istanze verrà ripetuto 5 volte per ottenenere la media dei tempi di esecuzione di ogni combinazione di parametri

In [ ]:
results = []

#SSPMF Model
print("SSPMF Model")
for index,row in scalability_df.iterrows():

    print("Instance: ", row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
        SSPMF_model.optimize()
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    results.append(["SSPMF",num_jobs, num_tools, magazine_capacity, mean_time])

print("-----------------------------------")
#JGSMF Model
print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_phases_3(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600, None)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, mean_time])

results_df = pd.DataFrame(results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])

In [14]:
results_df
results_df.to_csv("../results/results.csv", index=False)

Per dei risultati più attendibili, in quanto l'esecuzione sopra è stata discretamente veloce, gli esperimenti verranno ora ripetuti considerando 3 diverse istanze per combinazione di parametri ed eseguendone poi la mediana sulla media dei tempi, prendendo così il valore intermedio

In [ ]:
new_scalability_df = df.groupby(['num_tools', 'magazine_capacity']).head(3)

In [ ]:
new_results_df = []
# SSPMF Model
print("SSPMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            SSPMF_model = SSPMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
            SSPMF_model.optimize()
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        mean_time = pd.Series(iteration_times).mean()
        same_iteration_times.append(mean_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["SSPMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

print("-----------------------------------")
# JGSMF Model
print("JGSMF Model")
for (num_tools, magazine_capacity), group in new_scalability_df.groupby(['num_tools', 'magazine_capacity']):
    same_iteration_times = []
    print("Group ", num_tools, magazine_capacity)

    for index, row in group.iterrows():
        print("Instance", row['filename'])
        num_jobs = row['num_jobs']
        job_tools_requirements = row['job_tools']

        jobs = [x for x in range(1, num_jobs + 1)]
        tools = [x for x in range(1, num_tools + 1)]

        iteration_times = []

        for i in range(0, 5):
            start_time = datetime.now()
            solution_output = solve_with_phases_3(jobs, tools, magazine_capacity, job_tools_requirements, time_limit=3600, None)
            if solution_output is not None:
                job_order, switches = solution_output
            end_time = datetime.now()
            elapsed_time = (end_time - start_time).total_seconds()
            iteration_times.append(elapsed_time)

        mean_time = pd.Series(iteration_times).mean()
        same_iteration_times.append(mean_time)

    overall_median_time = pd.Series(same_iteration_times).median()
    new_results_df.append(["JGSMF", num_jobs, num_tools, magazine_capacity, overall_median_time])

new_results_df = pd.DataFrame(new_results_df, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])


In [15]:
new_results_df
new_results_df.to_csv("../results/new_results.csv", index=False)

Qui vengono eseguiti gli esperimenti sulle 4 diverse implementazioni di JGSMF, ottenute aggiungendo di volta in volta diversi i cut constraint. Nello specifico:
- JGSMF -> no symmetry/performance cuts
- JGSMF_1 -> symmetry cuts
- JGSMF_2 -> performance cuts senza cliques
- JGSMF_3 -> aggiunte anche le cliques

Inoltre trattandosi di tanti esperimenti, si ottimizzerà solamente un instanza per combinazione di parametri sempre dal cluster A (Tabela 4)

In [ ]:
JGSMF_results = []

print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_phases_0(jobs, tools, magazine_capacity, job_tools_requirements,3600, num_bins)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 1")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_phases_1(jobs, tools, magazine_capacity, job_tools_requirements,3600, num_bins)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_1",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 2")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_phases_2(jobs, tools, magazine_capacity, job_tools_requirements,3600, num_bins)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_2",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 3")

for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_phases_3(jobs, tools, magazine_capacity, job_tools_requirements, 3600, num_bins)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_3",num_jobs, num_tools, magazine_capacity, mean_time])

JGSMF_results_df = pd.DataFrame(JGSMF_results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])


In [ ]:
JGSMF_results = []

print("JGSMF Model")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_constant_bins_0(jobs, tools, magazine_capacity, job_tools_requirements, num_bins,3600)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 1")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_constant_bins_1(jobs, tools, magazine_capacity, job_tools_requirements, num_bins, 3600)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_1",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 2")
for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_constant_bins_2(jobs, tools, magazine_capacity, job_tools_requirements, num_bins, 3600)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_2",num_jobs, num_tools, magazine_capacity, mean_time])
print("-----------------------------------")
print("JGSMF Model 3")

for index,row in scalability_df.iterrows():

    print("Instance:" , row["filename"])
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    same_iteration_times = []
    mean_time = 0
    
    num_bins = math.ceil(num_jobs/2)

    for i in range(0, 5):
        start_time = datetime.now()
        solution_output = solve_with_constant_bins_3(jobs, tools, magazine_capacity, job_tools_requirements, num_bins, 3600)
        if solution_output is not None:
            job_order, switches = solution_output
        end_time = datetime.now()
        elapsed_time = (end_time - start_time).total_seconds()
        same_iteration_times.append(elapsed_time)
        
    mean_time = pd.Series(same_iteration_times).mean()
    JGSMF_results.append(["JGSMF_3",num_jobs, num_tools, magazine_capacity, mean_time])

JGSMF_results_df = pd.DataFrame(JGSMF_results, columns=["Model", "num_jobs", "num_tools", "magazine_capacity", "time"])


Instance: L20-7.txt
Set parameter Username
Set parameter LicenseID to value 2588882
Academic license - for non-commercial use only - expires 2025-11-22
Instance: L16-8.txt
Instance: L6-1.txt
Instance: L9-4.txt
Instance: L4-3.txt
Instance: L12-5.txt
Instance: L1-2.txt
Instance: L30-9.txt
Instance: L27-2.txt


In [ ]:
JGSMF_results_df
JGSMF_results_df.to_csv("../results/JGSMF_results.csv", index=False)

### Grafici e tabelle dei risultati

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt